In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import itertools
import re
import collections
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()


In [2]:
df = pd.read_parquet("df.parquet")

In [3]:
df

,record_id,product,description,sub_category,brand,company,company_parent,market_country,store_name,store_type,import_status,unit_price,percentile,premium,doc_vec,description_token,description_wv
0,7146579,Nocturnal Nectar Bio-Fermented Facial Essence,"Named after an English wild-flower meadow, Aml...",Face/Neck Care,Amly,Amly Botanicals,Amly Botanicals,Germany,None,None,Imported,402266.00,1.000000,1,"[-0.33388913, 2.1469963, 0.96274215, -0.878904...","[name, english, wild, flower, meadow, amli, de...","[[-0.051982604, -0.08290577, 0.3159616, 0.0085..."
1,6444419,Micro Needling Pimple Patches,Vice Reversa aims to focus on targeted deliver...,Face/Neck Care,Vice Reversa,A Beauty Story,A Beauty Story,UK,Victoria Health,Internet/Mail Order,Imported,265200.00,0.999939,1,"[0.59363365, 2.8043451, 0.6298298, -1.0089695,...","[vice, reversa, aim, focu, target, deliveri, s...","[[0.3694772, -0.57862556, -2.0269456, -0.64512..."
2,6993711,Capsules Protection Totale Face Treatment Oil,The Jean d'Arcel Renovar collection is said to...,Face/Neck Care,Jean d'Arcel Renovar,Jean d'Arcel Cosmétique,Jean d'Arcel Cosmétique,Germany,Feel-Beauty.de,Internet/Mail Order,Not imported,242950.00,0.999878,1,"[0.39346665, -1.5720978, 0.32630467, -0.300545...","[jean, arcel, renovar, collect, said, featur, ...","[[0.8480468, -0.055171926, -0.54320484, 0.1345..."
3,6444421,Micro Needling Plumping Patches,Vice Reversa aims to focus on targeted deliver...,Face/Neck Care,Vice Reversa,A Beauty Story,A Beauty Story,UK,Victoria Health,Internet/Mail Order,Imported,232350.00,0.999817,1,"[0.16396545, 2.3078039, 1.7311268, -0.7228534,...","[vice, reversa, aim, focu, target, deliveri, s...","[[0.3694772, -0.57862556, -2.0269456, -0.64512..."
4,6511823,"Dermatite, Psoriasis, Eczema CBD (5%) Skin Rep...",You & Oil products for skin and hair are creat...,Body Care,You & Oil Phyto Derma Therapy,JSC Biokosmetikos,JSC Biokosmetikos,Germany,Savue,Internet/Mail Order,nan,63360.00,0.999757,1,"[1.53203, 1.2907141, -0.31864712, -1.9782542, ...","[oil, product, skin, hair, creat, exclus, ecol...","[[0.9449418, -0.19407645, 0.33627698, -1.56030..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3287,7111519,Body Lotion,Edeka Elkos Body Body Lotion has been relaunch...,Body Care,Edeka Elkos Body,Edeka Zentrale,Edeka Zentrale,Germany,Diska,Supermarket,nan,0.25,0.000122,0,"[-1.3947046, 0.4628543, -0.00500729, 0.6803158...","[edeka, elko, bodi, bodi, lotion, relaunch, ne...","[[0.16146915, 0.81466347, 0.26688766, 1.377136..."
3288,6703603,Special Natural Clay Powder,Seara Argila Natural Especial em Pó (Special N...,Body Care,Seara,Seara Produtos Naturais,Seara Produtos Naturais,Portugal,Continente,Mass Merchandise/Hypermarket,nan,0.25,0.000122,0,"[1.0026945, -0.29684862, 0.1989893, 0.28999498...","[seara, argila, natur, especi, em, pó, special...","[[0.24507636, 0.19971885, 0.1813712, 0.0929433..."
3289,6331657,Coenzyme Q10 Body Milk,Amalfi Skin Care Leite Corporal Coenzima Q10 (...,Body Care,Amalfi Skin Care,Quimi Romar,Quimi Romar,Portugal,Jumbo Auchan,Mass Merchandise/Hypermarket,Imported,0.25,0.000122,0,"[-0.060306523, 0.06860288, 0.31759515, 0.22624...","[amalfi, skin, care, leit, corpor, coenzima, c...","[[-1.9248099, 0.80903107, 0.48631546, 0.015191..."
3290,6855981,Body Milk,Amalfi Skin Care Rosa Mosqueta Leche Corporal ...,Body Care,Amalfi Skin Care Rosa Mosqueta,Quimi Romar,Quimi Romar,Spain,Carrefour,Supermarket,Not imported,0.23,0.000061,0,"[0.009814276, 0.013060865, 0.09264118, 0.71502...","[amalfi, skin, care, rosa, mosqueta, lech, cor...","[[-1.9248099, 0.80903107, 0.48631546, 0.015191..."


In [10]:
Premium_NonPremium = df['premium']
Doc_Vec = df['doc_vec']

In [39]:
Doc_Vec_array = np.array(Doc_Vec.values.tolist())

In [40]:
#Split dataset into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Doc_Vec_array, Premium_NonPremium, test_size=0.2, random_state=100)

In [41]:
X_train

array([[-1.33352196e+00,  2.40045171e-02, -1.66448310e-01, ...,
        -7.33567059e-01,  3.27428937e-01, -3.71708602e-01],
       [-4.39466625e-01,  1.82640761e-01,  1.06200898e+00, ...,
         2.69202024e-01, -1.33528829e+00, -7.87972569e-01],
       [ 8.49326432e-01,  9.40741003e-01, -1.84730247e-01, ...,
        -2.49549642e-01, -2.94388771e+00, -1.54712903e+00],
       ...,
       [-7.08502352e-01,  4.72000152e-01,  7.35374331e-01, ...,
         5.68219900e-01,  4.98221248e-01,  4.93306145e-02],
       [-1.30063444e-01, -3.63060951e-01, -6.21677160e-01, ...,
        -1.21646285e-01,  2.06929818e-01,  1.68720179e-03],
       [ 6.70225024e-01,  4.34521288e-01,  5.12650251e-01, ...,
         4.96449679e-01,  5.39018869e-01,  4.81423527e-01]], dtype=float32)

In [46]:
X_train.shape

(2633, 300)

In [42]:
len(X_train)

2633

In [43]:
len(y_train)

2633

In [55]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\we704\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_log = logreg.predict(X_test)

In [57]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred_log))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred_log, average='weighted')))

Testing accuracy 0.9423368740515933
Testing F1 score: 0.9423419238275568


In [44]:
from sklearn.naive_bayes import MultinomialNB

baseline_model = MultinomialNB()

In [45]:
baseline_model.fit(X_train, y_train)

ValueError: Input X must be non-negative